In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

25/03/04 12:34:46 WARN Utils: Your hostname, ubuntu2004 resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/03/04 12:34:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/04 12:34:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-04 12:36:08--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.160, 3.164.82.112, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  4.27MB/s    in 15s     

2025-03-04 12:36:23 (4.20 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [7]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [ ]:
df_yellow.head(5)

[Row(VendorID=2, tpep_pickup_datetime=datetime.datetime(2024, 9, 30, 20, 30, 44), tpep_dropoff_datetime=datetime.datetime(2024, 9, 30, 20, 48, 26), passenger_count=1, trip_distance=3.0, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=246, payment_type=1, fare_amount=18.4, extra=1.0, mta_tax=0.5, tip_amount=1.5, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=24.9, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 9, 30, 20, 12, 20), tpep_dropoff_datetime=datetime.datetime(2024, 9, 30, 20, 25, 25), passenger_count=1, trip_distance=2.2, RatecodeID=1, store_and_fwd_flag='N', PULocationID=48, DOLocationID=236, payment_type=1, fare_amount=14.2, extra=3.5, mta_tax=0.5, tip_amount=3.8, tolls_amount=0.0, improvement_surcharge=1.0, total_amount=23.0, congestion_surcharge=2.5, Airport_fee=0.0),
 Row(VendorID=1, tpep_pickup_datetime=datetime.datetime(2024, 9, 30, 20, 4, 46), tpep_dropoff_datetime=datetime.datet

In [ ]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [8]:
#Repartition the Dataframe to 4 partitions and save it to parquet
df_yellow = df_yellow.repartition(4)
df_yellow.write.parquet('data/pq/yellow/2024/10/', mode='overwrite')

In [9]:
#2. What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)
!ls -lh 'data/pq/yellow/2024/10/'

total 100M
-rw-r--r-- 1 ubuntu ubuntu 25M Mar  4 13:14 part-00000-20799436-5718-46af-8096-6010878d2646-c000.snappy.parquet
-rw-r--r-- 1 ubuntu ubuntu 25M Mar  4 13:14 part-00001-20799436-5718-46af-8096-6010878d2646-c000.snappy.parquet
-rw-r--r-- 1 ubuntu ubuntu 25M Mar  4 13:14 part-00002-20799436-5718-46af-8096-6010878d2646-c000.snappy.parquet
-rw-r--r-- 1 ubuntu ubuntu 25M Mar  4 13:14 part-00003-20799436-5718-46af-8096-6010878d2646-c000.snappy.parquet
-rw-r--r-- 1 ubuntu ubuntu   0 Mar  4 13:14 _SUCCESS


In [ ]:
#3. How many taxi trips were there on the 15th of October?
from pyspark.sql import functions as F
df_yellow = spark.read.parquet('data/pq/yellow/2024/10')

In [35]:
df_yellow = df_yellow \
    .withColumn("pickup_date", F.to_date(df_yellow.tpep_pickup_datetime)) \
    .withColumn("dropoff_date", F.to_date(df_yellow.tpep_dropoff_datetime))

df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|dropoff_date|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+
|       2| 2024-10-12 00:16:59|  2024-10-12 00:24:37|              2|         2.07|         1|                 N

In [43]:
df_yellow.groupBy('pickup_date').count().filter(df_yellow.pickup_date == '2024-10-15').show()

+-----------+------+
|pickup_date| count|
+-----------+------+
| 2024-10-15|128909|
+-----------+------+



In [47]:
df_yellow = df_yellow \
    .withColumn("trip_duration", (F.unix_timestamp(df_yellow.tpep_dropoff_datetime) - F.unix_timestamp(df_yellow.tpep_pickup_datetime))/3600) 

df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|dropoff_date|       trip_duration|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+--------------------+
|       2| 2024-10-12 00:16:59|  2024-10-12 00:24

In [50]:
df_yellow.sort(F.desc("trip_duration")).limit(1).show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|pickup_date|dropoff_date|     trip_duration|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-----------+------------+------------------+
|       2| 2024-10-16 09:03:49|  2024-10-23 03:40:53|  

In [52]:
#6 Least frequent pickup location zone

#Load the zone lookup data into a temp view in Spark:
#wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-04 15:33:24--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.160, 3.164.82.40, 3.164.82.112, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0.001s  

2025-03-04 15:33:24 (12.8 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [53]:
zone_lookup = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

zone_lookup.limit(5).show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [70]:
zone_lookup.registerTempTable('zone_data')
df_yellow.registerTempTable('yellow_data')

q_result = spark.sql("""
SELECT zone_data.Zone, count(1) as trips_number
FROM yellow_data
INNER JOIN zone_data
ON zone_data.LocationID = yellow_data.PULocationID
GROUP BY zone_data.Zone
ORDER BY trips_number
""").limit(5).show()

/home/ubuntu/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+--------------------+------------+
|                Zone|trips_number|
+--------------------+------------+
|Governor's Island...|           1|
|       Rikers Island|           2|
|       Arden Heights|           2|
|         Jamaica Bay|           3|
| Green-Wood Cemetery|           3|
+--------------------+------------+

